In [15]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
from tqdm import tqdm
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate 
from tensorflow.keras.layers import Input, Add, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import clear_output
%matplotlib inline

from IPython.display import HTML
from base64 import b64encode

In [16]:
# Load directories
train_data_dir = r"C:\Users\NA\Documents\Free Code Camp\training\image_2"
train_gt_dir = r"C:\Users\NA\Documents\Free Code Camp\training\gt_image_2"

test_data_dir = r"C:\Users\NA\Documents\Free Code Camp\testing"

In [17]:
# Load and sort image and mask paths
image_paths = sorted(tf.io.gfile.glob(train_data_dir + "*.png"))
mask_paths = sorted(tf.io.gfile.glob(train_gt_dir + "*.png"))  # Ensuring correct order

In [18]:
# Number of training examples
TRAINSET_SIZE = int(len(os.listdir(train_data_dir)) * 0.8)
print(f"Number of Training Examples: {TRAINSET_SIZE}")

VALIDSET_SIZE = int(len(os.listdir(train_data_dir)) * 0.1)
print(f"Number of Validation Examples: {VALIDSET_SIZE}")

TESTSET_SIZE = int(len(os.listdir(train_data_dir)) - TRAINSET_SIZE - VALIDSET_SIZE)
print(f"Number of Testing Examples: {TESTSET_SIZE}")


Number of Training Examples: 231
Number of Validation Examples: 28
Number of Testing Examples: 30


In [19]:
# Initialize Constants
IMG_SIZE = 128
N_CHANNELS = 3
N_CLASSES = 1
SEED = 123

In [24]:
# Function to load image-mask pairs
def parse_image(img_path, mask_path):
    # Read and decode image
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)  # PNG format
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
    
    # Read and decode mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)  # PNG format
    
    # Define road & non-road classes
    road_label = np.array([255, 0, 255])
    
    # Convert to binary mask
    mask = tf.experimental.numpy.all(mask == road_label, axis=-1)
    mask = tf.cast(mask, tf.uint8)
    mask = tf.expand_dims(mask, axis=-1)  # Ensure shape is (H, W, 1)
    
    return image, mask

In [25]:
# Create TensorFlow dataset
image_ds = tf.data.Dataset.from_tensor_slices(image_paths)
mask_ds = tf.data.Dataset.from_tensor_slices(mask_paths)

In [26]:
# Zip datasets to keep images and masks aligned
dataset = tf.data.Dataset.zip((image_ds, mask_ds))
dataset = dataset.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)


TypeError: in user code:

    File "C:\Users\NA\AppData\Local\Temp\ipykernel_17840\3915343635.py", line 4, in parse_image  *
        image = tf.io.read_file(img_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


In [23]:
# Generate dataset variables
all_dataset = tf.data.Dataset.list_files(train_data_dir + "*.png", seed=SEED)
all_dataset = all_dataset.map(parse_image)

train_dataset = all_dataset.take(TRAINSET_SIZE + VALIDSET_SIZE)
val_dataset = train_dataset.skip(TRAINSET_SIZE)
train_dataset = train_dataset.take(TRAINSET_SIZE)
test_dataset = all_dataset.skip(TRAINSET_SIZE + VALIDSET_SIZE)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: C:\\Users\\NA\\Documents\\Free Code Camp\\training\\image_2*.png'